In [14]:
#imports
import os
import shutil
import subprocess

In [15]:
DATASET_1 = 'Data_1'
DATASET_2 = 'Data_2'
DATASETS = [DATASET_2]

SCENARIO_1 = 'Ulds_scenario_1'
SCENARIO_2a = 'Ulds_scenario_2a'
SCENARIO_2b = 'Ulds_scenario_2b'
SCENARIOS = [SCENARIO_1, SCENARIO_2a, SCENARIO_2b]

ITEMS_BELOW_LEVELS = [20]

PATH = './Data'

In [16]:
def createOutputFolderIfNotExists(path):
    isExist = os.path.exists(path)
    if isExist:
        shutil.rmtree(path)
        os.makedirs(path)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(path)
    print("The new directory is created!")

In [17]:
def copyFile(file, destination):
    shutil.copy(file, destination)

In [18]:
def copyFilesInFolder(source_folder, destination_folder):
    # fetch all files
    for file_name in os.listdir(source_folder):
        # construct full file path
        source = source_folder + file_name
        destination = destination_folder + file_name
        # copy only files
        if os.path.isfile(source):
            copyFile(source, destination)
            print('copied', file_name)    

In [19]:
import json

def readJson(filePathImport):
    #Read file in
    with open(filePathImport) as json_file:
        json_file = json.load(json_file)
        return json_file

In [20]:
def writeJson(filePathExport, jsonFile):
    with open(filePathExport, 'w', encoding='utf-8') as f:
        json.dump(jsonFile, f, ensure_ascii=False, indent=4)

In [21]:
def makeFolderStructure(path_to_benchmark_scenario):
    createOutputFolderIfNotExists(path_to_benchmark_scenario + "/todo")
    for level in ITEMS_BELOW_LEVELS:
        createOutputFolderIfNotExists(path_to_benchmark_scenario + "/selected/" + str(level))

In [22]:
def executeJavaTool(itemsbelow, input_path, output_path):
    subprocess.run(['java', '-jar', 'Select-Files.jar',
                    '--itemsbelow', str(itemsbelow), input_path, output_path], capture_output=True)

In [23]:
def clip_jobs_at_item_level(n_items = 19, path_to_todo = "", path_to_selected = ""):
    for file_name in os.listdir(path_to_todo):
        json = readJson(path_to_todo + "/" + file_name)
        json_new = json
        json_new["ulds"][0]["placedItems"] = json_new["ulds"][0]["placedItems"][:n_items]
        writeJson(path_to_selected + "/" + file_name, json_new)

In [24]:
def makeArchive(path_to_selected, level, zipped_file_name):
    subprocess.run(['tar', '-czvf', zipped_file_name, "-C", 
                    path_to_selected + str(level), '.', '--exclude=*.DS_Store'], capture_output=True)

In [25]:
def prepare(dataset, scenario, path_to_dataset):
    path_to_benchmark_scenario = path_to_dataset + "/4_Benchmark" + "/" + scenario 
    path_to_jobs_full = path_to_dataset + "/2_AeJobs" + "/" + scenario + "/"
    path_to_todo = path_to_benchmark_scenario + "/todo/"
    path_to_selected = path_to_benchmark_scenario + "/selected/"
    
    makeFolderStructure(path_to_benchmark_scenario)
    
    # copy AeJobs to /todo
    copyFilesInFolder(path_to_jobs_full, path_to_todo)
    
    for level in ITEMS_BELOW_LEVELS:
        zipped_file_name = path_to_selected + dataset + "_" + scenario + "_" + str(level) + ".tgz"
        
        # execute java tool to seperate ULDs based on number of placed items 
        executeJavaTool(level, path_to_todo, path_to_selected + str(level))
        
        # clip other jobs s.t. they only contain at max n_items
        clip_jobs_at_item_level(level, path_to_todo, path_to_selected + str(level))
        
        # make archive out of folders in /selected
        makeArchive(path_to_selected, level, zipped_file_name)

In [26]:
for dataset in DATASETS:
    for scenario in SCENARIOS:
        path_to_dataset = PATH + "/" + dataset
        prepare(dataset, scenario, path_to_dataset)

The new directory is created!
The new directory is created!
copied Data_2_Ulds_scenario_1_H129_Class_1_Instance_15_ULD_0.json
copied Data_2_Ulds_scenario_1_H144_Class_3_Instance_0_ULD_7.json
copied Data_2_Ulds_scenario_1_H3_Class_1_Instance_30_ULD_0.json
copied Data_2_Ulds_scenario_1_H7_Class_3_Instance_0_ULD_2.json
copied Data_2_Ulds_scenario_1_H25_Class_3_Instance_15_ULD_2.json
copied Data_2_Ulds_scenario_1_H153_Class_3_Instance_0_ULD_0.json
copied Data_2_Ulds_scenario_1_H42_Class_3_Instance_45_ULD_2.json
copied Data_2_Ulds_scenario_1_H39_Class_3_Instance_30_ULD_11.json
copied Data_2_Ulds_scenario_1_H42_Class_3_Instance_30_ULD_10.json
copied Data_2_Ulds_scenario_1_H44_Class_3_Instance_30_ULD_0.json
copied Data_2_Ulds_scenario_1_H102_Class_3_Instance_15_ULD_5.json
copied Data_2_Ulds_scenario_1_H91_Class_3_Instance_0_ULD_9.json
copied Data_2_Ulds_scenario_1_H151_Class_3_Instance_30_ULD_12.json
copied Data_2_Ulds_scenario_1_H128_Class_2_Instance_0_ULD_0.json
copied Data_2_Ulds_scenario_